In [7]:
%load_ext autoreload
%autoreload 2

import pyreadstat as prs

raw_df, meta = prs.read_sas7bdat("../dat/GSS_sas/gss7222_r3.sas7bdat")

In [9]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')

from transform_df_to_our_standard import transform_dataframe, make_variable_summary
from corr_networks import corr_mat_to_partial_corr_mat, cov_mat_to_regularized_partial_corr
from triads import count_triads

In [56]:
df = transform_dataframe(raw_df)

c:\Users\vicvi\BeliefNetworkEvo\nbk\../src\transform_df_to_our_standard.py:466: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed_df['PRES12_DIDNT_VOTE'] = df['PRES12'].map(category_map_D)
c:\Users\vicvi\BeliefNetworkEvo\nbk\../src\transform_df_to_our_standard.py:467: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed_df['PRES12_DONT_KNOW'] = df['PRES12'].map(DONT_KNOW_map)
c:\Users\vicvi\BeliefNetworkEvo\nbk\../src\transform_df_to_our_standard.py:469: PerformanceWarning: DataFrame is highly fragmented.  This is u

In [ ]:
cols = df.columns.to_list()
"RACOPEN" in cols

In [44]:
vars_to_use = ["PARTYID","POLVIEWS","NATSPAC","NATENVIR","NATHEAL","NATCITY","NATCRIME","NATDRUG","NATEDUC","NATRACE","NATARMS",
"NATAID","NATFARE","NATROAD","NATSOC","NATMASS","NATPARK","NATCHLD","NATSCI","NATENRGY","NATSPACY","NATENVIY","NATHEALY","NATCITYY","NATCRIMY","NATDRUGY","NATEDUCY",
"NATRACEY","NATARMSY","NATAIDY","NATFAREY","EQWLTH","SPKATH","COLATH","LIBATH","SPKRAC","COLRAC","LIBRAC","SPKCOM","COLCOM","LIBCOM","SPKMIL","COLMIL","LIBMIL","SPKHOMO",
"COLHOMO","LIBHOMO","SPKMSLM","COLMSLM","LIBMSLM","CAPPUN","GUNLAW","COURTS","GRASS","ATTEND","RELITEN","POSTLIFE","PRAYER","AFFRMACT","WRKWAYUP","HELPFUL",
"FAIR","TRUST","CONFINAN","CONBUS","CONCLERG","CONEDUC","CONFED","CONLABOR","CONPRESS","CONMEDIC","CONTV","CONJUDGE","CONSCI","CONLEGIS","CONARMY","GETAHEAD","FEPOL","ABDEFECT","ABNOMORE","ABHLTH","ABPOOR","ABRAPE","ABSINGLE","ABANY","SEXEDUC","DIVLAW","PREMARSX","TEENSEX","XMARSEX","HOMOSEX","PORNLAW",
"SPANKING","LETDIE1","SUICIDE1","SUICIDE2","POLHITOK","POLABUSE","POLMURDR","POLESCAP","POLATTAK","NEWS","TVHOURS","FECHLD","FEPRESCH","FEFAM","RACDIF1","RACDIF2","RACDIF3",
"RACDIF4","HELPPOOR","MARHOMO", "RACOPEN", "HELPNOT", "HELPBLK"]

PRES_variables = [
    'VOTE68', 'VOTE72', 'VOTE76', 'VOTE80', 'VOTE84', 'VOTE88', 'VOTE92', 'VOTE96', 'VOTE00', 'VOTE04', 'VOTE08', 'VOTE12', 'VOTE16', 'VOTE20',
    'VOTE68_ELIGIBLE', 'VOTE72_ELIGIBLE', 'VOTE76_ELIGIBLE', 'VOTE80_ELIGIBLE', 'VOTE84_ELIGIBLE', 'VOTE88_ELIGIBLE', 'VOTE92_ELIGIBLE', 'VOTE96_ELIGIBLE', 'VOTE00_ELIGIBLE', 'VOTE04_ELIGIBLE', 'VOTE08_ELIGIBLE', 'VOTE12_ELIGIBLE', 'VOTE16_ELIGIBLE', 'VOTE20_ELIGIBLE',
    'PRES68_HUMPHREY', 'PRES68_NIXON', 'PRES68_WALLACE', 'PRES68_OTHER', 'PRES68_REFUSED',
    'PRES72_MCGOVERN', 'PRES72_NIXON', 'PRES72_OTHER', 'PRES72_REFUSED', 'PRES72_WOULDNT_VOTE', 'PRES72_DONT_KNOW',
    'PRES76_CARTER', 'PRES76_FORD', 'PRES76_OTHER', 'PRES76_REFUSED', 'PRES76_NO_PRES_VOTE', 'PRES76_DONT_KNOW',
    'PRES80_CARTER', 'PRES80_REAGAN', 'PRES80_ANDERSON', 'PRES80_OTHER', 'PRES80_REFUSED', 'PRES80_DIDNT_VOTE', 'PRES80_DONT_KNOW',
    'PRES84_MONDALE', 'PRES84_REAGAN', 'PRES84_OTHER', 'PRES84_REFUSED', 'PRES84_NO_PRES_VOTE', 'PRES84_DONT_KNOW',
    'PRES88_BUSH', 'PRES88_DUKAKIS', 'PRES88_OTHER', 'PRES88_REFUSED', 'PRES88_NO_PRES_VOTE', 'PRES88_DONT_KNOW',
    'PRES92_CLINTON', 'PRES92_BUSH', 'PRES92_PEROT', 'PRES92_OTHER', 'PRES92_NO_PRES_VOTE', 'PRES92_DONT_KNOW',
    'PRES96_CLINTON', 'PRES96_DOLE', 'PRES96_PEROT', 'PRES96_OTHER', 'PRES96_DIDNT_VOTE', 'PRES96_DONT_KNOW',
    'PRES00_GORE', 'PRES00_BUSH', 'PRES00_NADER', 'PRES00_OTHER', 'PRES00_DIDNT_VOTE', 'PRES00_DONT_KNOW',
    'PRES04_KERRY', 'PRES04_BUSH', 'PRES04_NADER', 'PRES04_NO_PRES_VOTE', 'PRES04_DONT_KNOW',
    'PRES08_OBAMA', 'PRES08_MCCAIN', 'PRES08_OTHER', 'PRES08_DIDNT_VOTE', 'PRES08_DONT_KNOW',
    'PRES12_OBAMA', 'PRES12_ROMNEY', 'PRES12_OTHER', 'PRES12_DIDNT_VOTE', 'PRES12_DONT_KNOW',
    'PRES16_CLINTON', 'PRES16_TRUMP', 'PRES16_OTHER', 'PRES16_DIDNT_VOTE', 'PRES16_DONT_KNOW',
    'PRES20_BIDEN', 'PRES20_TRUMP', 'PRES20_OTHER', 'PRES20_DIDNT_VOTE', 'PRES20_DONT_KNOW',
    'IF68WHO_HUMPHREY', 'IF68WHO_NIXON', 'IF68WHO_WALLACE', 'IF68WHO_OTHER', 'IF68WHO_WLDNT_VT_RELIG', 'IF68WHO_DONT_KNOW',
    'IF72WHO_MCGOVERN', 'IF72WHO_NIXON', 'IF72WHO_OTHER', 'IF72WHO_REFUSED', 'IF72WHO_WOULDNT_VOTE', 'IF72WHO_WLDNT_VT_RELIG', 'IF72WHO_DONT_KNOW',
    'IF76WHO_CARTER', 'IF76WHO_FORD', 'IF76WHO_OTHER', 'IF76WHO_REFUSED', 'IF76WHO_WOULDNT_VOTE', 'IF76WHO_DONT_KNOW',
    'IF80WHO_CARTER', 'IF80WHO_REAGAN', 'IF80WHO_ANDERSON', 'IF80WHO_OTHER', 'IF80WHO_WOULDNT_VOTE', 'IF80WHO_REFUSED', 'IF80WHO_DONT_KNOW',
    'IF84WHO_MONDALE', 'IF84WHO_REAGAN', 'IF84WHO_OTHER', 'IF84WHO_WOULDNT_VOTE', 'IF84WHO_DONT_KNOW',
    'IF88WHO_DUKAKIS', 'IF88WHO_BUSH', 'IF88WHO_OTHER', 'IF88WHO_DONT_KNOW',
    'IF92WHO_CLINTON', 'IF92WHO_BUSH', 'IF92WHO_PEROT', 'IF92WHO_OTHER', 'IF92WHO_DONT_KNOW',
    'IF96WHO_CLINTON', 'IF96WHO_DOLE', 'IF96WHO_PEROT', 'IF96WHO_OTHER', 'IF96WHO_DONT_KNOW',
    'IF00WHO_GORE', 'IF00WHO_BUSH', 'IF00WHO_NADER', 'IF00WHO_OTHER', 'IF00WHO_DONT_KNOW',
    'IF04WHO_KERRY', 'IF04WHO_BUSH', 'IF04WHO_NADER', 'IF04WHO_DONT_KNOW',
    'IF08WHO_OBAMA', 'IF08WHO_MCCAIN', 'IF08WHO_OTHER', 'IF08WHO_DONT_KNOW',
    'IF12WHO_OBAMA', 'IF12WHO_ROMNEY', 'IF12WHO_OTHER', 'IF12WHO_DONT_KNOW',
    'IF16WHO_CLINTON', 'IF16WHO_TRUMP', 'IF16WHO_OTHER', 'IF16WHO_CANT_REMEMBER', 'IF16WHO_DONT_KNOW',
    'IF20WHO_BIDEN', 'IF20WHO_TRUMP', 'IF20WHO_OTHER', 'IF20WHO_CANT_REMEMBER', 'IF20WHO_DONT_KNOW'
]


# BROKEN_VARS = ["RELIG",'RACOPEN', 'HELPNOT', 'HELPBLK']
# Ommited from this list: "YEAR", "BALLOT", and "LETDIE1" (this one sucks!)


In [ ]:
from make_belief_network import make_belief_network
import networkx as nx
import numpy as np

variables_of_interest = vars_to_use

years = [2000, 2002, 2004, 2006, 2008, 2010, 2012]
thresholds = [0, 0.2, 0.3, 0.5, 0.6, 0.7]

outputs = {year: {partial: {} for partial in [True, False]} for year in years}

for year in [2000, 2002, 2004, 2006, 2008, 2010, 2012]:
    for partial in [True, False]:
        for threshold in thresholds:
            BN, variables_list, correlation_matrix = make_belief_network(df, variables_of_interest, [year], method="spearman", is_partial=partial, 
                                                                threshold=None, sample_threshold=threshold)

            outputs[year][partial][threshold] = {"network": BN, "variables": variables_list, "corr_mat": correlation_matrix}

In [ ]:
partial = True
for year in years:
    for threshold in thresholds:
        # for partial in [True, False]:
            correlation_matrix = outputs[year][partial][threshold]["corr_mat"]
            BN = outputs[year][partial][threshold]["network"]
            variables_list = outputs[year][partial][threshold]["variables"]

            print(f"{year}, partial corrs: {partial}, threshold: {threshold}")

            correlation_matrix_unique = correlation_matrix[np.triu_indices(correlation_matrix.shape[0], k=1)]

            # Print the max and min correlation values
            print(f"Max correlation: {np.max(correlation_matrix_unique)}")
            print(f"Min correlation: {np.min(correlation_matrix_unique)}")

            # Print num nodes and edges in graph
            print(f"Number of nodes: {len(BN.nodes)}")
            print(f"Number of edges: {len(BN.edges)}")
        
            """
            # Save the graph as graphml into the folder "outputs"
            # make the name include the years of interest
            years_name = "_".join([str(year) for year in years_of_interest])
            # filename = f"{years_name}.graphml"
            filename = "asdasd.graphml"
            nx.write_graphml(BN, f"out/{filename}")
            """

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows without truncation

print(df.loc[(df["YEAR"] == 2002) & df["CONEDUC"].isna() & ((df["BALLOT"] == 2.0) | (df["BALLOT"] == 3.0))].notna().any(axis=0))

In [57]:
filled_vars, percent_filled, partially_complete = make_variable_summary(df)

In [52]:
filled_vars.shape

(75, 303)

In [53]:
percent_filled.shape

(75, 303)

In [58]:
nonvoting_percent_filled = percent_filled.drop(PRES_variables, axis=1)
nonvoting_percent_filled.to_csv("variable_completion.csv")

In [45]:
[coord for coord in partially_complete if coord[1] not in PRES_variables]

[((1988.0, 1.0), 'NATENVIR'),
 ((1988.0, 2.0), 'NATENVIR'),
 ((1988.0, 3.0), 'NATENVIR'),
 ((1989.0, 1.0), 'NATENVIR'),
 ((1989.0, 2.0), 'NATENVIR'),
 ((1989.0, 3.0), 'NATENVIR'),
 ((1990.0, 1.0), 'NATENVIR'),
 ((1990.0, 2.0), 'NATENVIR'),
 ((1990.0, 3.0), 'NATENVIR'),
 ((1991.0, 1.0), 'NATENVIR'),
 ((1991.0, 2.0), 'NATENVIR'),
 ((1991.0, 3.0), 'NATENVIR'),
 ((1993.0, 1.0), 'NATENVIR'),
 ((1993.0, 2.0), 'NATENVIR'),
 ((1993.0, 3.0), 'NATENVIR'),
 ((1994.0, 1.0), 'NATENVIR'),
 ((1994.0, 1.0), 'AFFRMACT'),
 ((1994.0, 1.0), 'WRKWAYUP'),
 ((1994.0, 2.0), 'NATENVIR'),
 ((1994.0, 2.0), 'AFFRMACT'),
 ((1994.0, 2.0), 'WRKWAYUP'),
 ((1994.0, 3.0), 'NATENVIR'),
 ((1996.0, 1.0), 'NATENVIR'),
 ((1996.0, 2.0), 'NATENVIR'),
 ((1996.0, 2.0), 'RACOPEN'),
 ((1996.0, 3.0), 'NATENVIR'),
 ((1996.0, 3.0), 'RACOPEN'),
 ((1996.0, 3.0), 'FECHLD'),
 ((1996.0, 3.0), 'FEPRESCH'),
 ((1996.0, 3.0), 'FEFAM'),
 ((1998.0, 1.0), 'NATENVIR'),
 ((1998.0, 1.0), 'TRUST'),
 ((1998.0, 2.0), 'NATENVIR'),
 ((1998.0, 3.0), 'NA

In [28]:
data = {
    'A': [1.0, np.nan, 3.0, 4.0],
    'B': [5.0, 6.0, np.nan, 8.0],
    'C': [np.nan, 10.0, 11.0, 12.0]
}

df = pd.DataFrame(data)

In [29]:
df.notna()

,A,B,C
0,True,True,False
1,False,True,True
2,True,False,True
3,True,True,True


In [ ]:
partially_complete

In [ ]:
years_in_set

In [ ]:
import networkx as nx
from itertools import combinations

def find_triads(G, threshhold=0):
    nodes = G.nodes()
    triads = list(combinations(nodes, 3))
    positive_triads = []
    negative_triads = []

    for triad in triads:
        edges = [(triad[0], triad[1]), (triad[1], triad[2]), (triad[0], triad[2])]
        weights = [G[u][v]['weight'] for u, v in edges if G.has_edge(u, v)]

        # Ensure all edges exist, otherwise skip
        if len(weights) == 3:
            if abs(weights[0]) > threshhold and abs(weights[1]) > threshhold and abs(weights[2]) > threshhold:
            
                product = weights[0] * weights[1] * weights[2]
                if product > 0:
                    positive_triads.append(triad)
                elif product < 0:
                    negative_triads.append(triad)

    return positive_triads, negative_triads

# Create graph and find the triads, and print the number of triads
G = BN
positive_triads, negative_triads = find_triads(G, threshhold=0.2)
print(f"Number of balanced triads: {len(positive_triads)}")
print(f"Number of unbalanced triads: {len(negative_triads)}")

# Print the triads
print("Balanced triads:", positive_triads)
print("Unbalanced triads:", negative_triads)

In [ ]:
# Print the histogram of the correlation matrix
import matplotlib.pyplot as plt
plt.hist(correlation_matrix.flatten(), bins=100)
plt.xlabel("Correlation coefficient")
plt.ylabel("Frequency")
plt.title("Histogram of correlation coefficients")

plt.yscale('log')
plt.show()
plt.show()


In [14]:
measurements = {dim: {alpha: {} for alpha in [0, 0.1, 0.2]} for dim in [50, 100, 200]}
for dim in [50, 100, 200]:
    for alpha in [0, 0.1, 0.2]:
        neg_counts = []
        pos_counts = []
        for i in range(20):
            random_matrix = random_mat = 2 * np.random.rand(dim, dim) - 1
            random_cov_mat = np.dot(random_matrix, random_matrix.T)

            std_deviations = np.sqrt(np.diag(random_cov_mat))
            random_corr_mat = random_cov_mat / np.outer(std_deviations, std_deviations)

            random_partial_corr_mat = cov_mat_to_regularized_partial_corr(random_corr_mat, alpha=alpha)

            positive_count, negative_count = count_triads(ra)
            
            neg_counts.append(negative_count / (negative_count + positive_count))
            pos_counts.append(positive_count / (negative_count + positive_count))
        
        print(f"{dim}, {alpha}: negatives: {sum(neg_counts) / len(neg_counts)}, positives: {sum(pos_counts) / len(pos_counts)}")
        measurements[dim][alpha]["neg"] = sum(neg_counts) / len(neg_counts)
        measurements[dim][alpha]["pos"] =  sum(pos_counts) / len(pos_counts)

50, 0: negatives: 0.9085612244897957, positives: 0.09143877551020407
50, 0.1: negatives: 0.4066961724541849, positives: 0.593303827545815
50, 0.2: negatives: 0.1466443268449485, positives: 0.8533556731550516
100, 0: negatives: 0.8615231910946198, positives: 0.13847680890538033
100, 0.1: negatives: 0.3687557270077323, positives: 0.6312442729922675
100, 0.2: negatives: 0.06392224461303408, positives: 0.9360777553869658
200, 0: negatives: 0.8667048119384804, positives: 0.13329518806151974
200, 0.1: negatives: 0.30971363922537154, positives: 0.6902863607746283


C:\Users\vicvi\AppData\Local\Temp\ipykernel_23160\3102570745.py:30: RuntimeWarning: invalid value encountered in scalar divide
  neg_counts.append(negative_count / (negative_count + positive_count))
C:\Users\vicvi\AppData\Local\Temp\ipykernel_23160\3102570745.py:31: RuntimeWarning: invalid value encountered in scalar divide
  pos_counts.append(positive_count / (negative_count + positive_count))
C:\Users\vicvi\AppData\Local\Temp\ipykernel_23160\3102570745.py:30: RuntimeWarning: invalid value encountered in scalar divide
  neg_counts.append(negative_count / (negative_count + positive_count))
C:\Users\vicvi\AppData\Local\Temp\ipykernel_23160\3102570745.py:31: RuntimeWarning: invalid value encountered in scalar divide
  pos_counts.append(positive_count / (negative_count + positive_count))
C:\Users\vicvi\AppData\Local\Temp\ipykernel_23160\3102570745.py:30: RuntimeWarning: invalid value encountered in scalar divide
  neg_counts.append(negative_count / (negative_count + positive_count))
C:\Us

200, 0.2: negatives: nan, positives: nan


In [ ]:
measurements